## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. They often do that by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will look at how you can extract BGP configuration and session status information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example-bgp"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

load_questions()

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### View BGP Configuration for ALL devices
Batfish makes BGP Configuration settings in the network easily accessible. Let's take a look at how you can retrieve the specific information you want. Let's start with configuration attributes of the BGP process on all devices running BGP.

In [3]:
# Get BGP process configuration information for ALL devices
bgp_config = bfq.bgpProcessConfiguration().answer().frame()
bgp_config

,Node,VRF,Router_ID,Route_Reflector,Multipath_Match_Mode,Multipath_EBGP,Multipath_IBGP,Neighbors,Tie_Breaker
0,as1border1,default,1.1.1.1,False,EXACT_PATH,True,True,"['1.10.1.1/32', '3.2.2.2/32', '5.6.7.8/32', '10.12.11.2/32']",ARRIVAL_ORDER
1,as2dist2,default,2.1.3.2,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",ARRIVAL_ORDER
2,as2border1,default,2.1.1.1,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '10.12.11.1/32']",ARRIVAL_ORDER
3,as2core2,default,2.1.2.2,True,EXACT_PATH,True,True,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",ARRIVAL_ORDER
4,as3border1,default,3.1.1.1,False,EXACT_PATH,True,True,"['3.10.1.1/32', '10.23.21.2/32']",ARRIVAL_ORDER
5,as3border2,default,3.2.2.2,False,EXACT_PATH,True,True,"['3.10.1.1/32', '10.13.22.1/32']",ARRIVAL_ORDER
6,as3core1,default,3.10.1.1,True,EXACT_PATH,True,True,"['3.1.1.1/32', '3.2.2.2/32']",ARRIVAL_ORDER
7,as2dist1,default,2.1.3.1,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",ARRIVAL_ORDER
8,as1border2,default,1.2.2.2,False,EXACT_PATH,True,True,"['1.10.1.1/32', '10.13.22.3/32', '10.14.22.4/32']",ARRIVAL_ORDER
9,as2border2,default,2.1.1.2,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '10.23.21.3/32']",ARRIVAL_ORDER


NOTE: Any router that has a `Cluster ID` set is configured to be a route-reflector for some of it's BGP peers

In [4]:
# wrapper function that will return the definition of a routing policy
#
# Input:
#    node: Name of the device
#    policy_names: List of policy names to retrieve
#    config_file_path: Directory where the `configs` folder resides
#
# Output:
#    policy_def: dictionary keyed on policy name that contains the actual config lines that define the policy
def get_bgp_policies(node, policy_names, config_file_path):
    
    import linecache
    from collections import defaultdict
    
    # This returns the list of all defined structures on `as2dept1`
    def_struct = bfq.definedStructures(nodes=node).answer().frame()
    
    policy_def = defaultdict(list)

    # Retrieving just the structures that are used as BGP import policies for the specific peer we are looking at

    for policy in policy_names:
        struct = def_struct[def_struct['Structure_Name']==policy]
        file = config_file_path + "/" + struct.Source_Lines.iloc[0].filename

        lines = struct.Source_Lines.iloc[0].lines

        text_line = []

        for line in lines:
            text_line.append(linecache.getline(file, line))
            
        policy_def[policy] = text_line
            
    return(policy_def)

Now let's drill into the configuration of a specific BGP session. Let's look at the sessions on `as2dept1`

In [5]:
# Get all of the BGP peer configuration for ALl devices
bgp_peer_config = bfq.bgpPeerConfiguration().answer().frame()
bgp_peer_config[bgp_peer_config['Node'].str.contains('as2dept1')]

,Node,VRF,Local_AS,Local_IP,Local_Interface,Remote_AS,Remote_IP,Route_Reflector_Client,Cluster_ID,Peer_Group,Import_Policy,Export_Policy,Send_Community,Is_Passive
3,as2dept1,default,65001,2.34.201.4,None,2,2.34.201.3,False,None,as2,['as2_to_dept'],['dept_to_as2'],True,False
17,as2dept1,default,65001,2.34.101.4,None,2,2.34.101.3,False,None,as2,['as2_to_dept'],['dept_to_as2'],True,False


`as2dept1` has 2 configured bgp peers. Let's take a look at the input and output route policies for the peer `2.24.101.3`

In [6]:
# as2dept1 BGP peer configuration that we want to dig into
peer_ip = '2.34.101.3'

peer_import_policy = bgp_peer_config[(bgp_peer_config['Node'].str.contains('as2dept1')) & (bgp_peer_config['Remote_IP']==peer_ip)].Import_Policy.iloc[0]
peer_export_policy = bgp_peer_config[(bgp_peer_config['Node'].str.contains('as2dept1')) & (bgp_peer_config['Remote_IP']==peer_ip)].Export_Policy.iloc[0]

In [7]:
import_policy_text = get_bgp_policies('as2dept1', peer_import_policy, SNAPSHOT_PATH)

for import_policy in import_policy_text:
    print("Import Policy: {}".format(import_policy))
    print("Definition: {}".format(import_policy_text[import_policy]))

Import Policy: as2_to_dept
Definition: ['route-map as2_to_dept permit 100\n', ' match community as2_community\n', ' set local-preference 350\n']


In [8]:
export_policy_text = get_bgp_policies('as2dept1', peer_export_policy, SNAPSHOT_PATH)

for export_policy in export_policy_text:
    print("Export Policy: {}".format(export_policy))
    print("Definition: {}".format(export_policy_text[export_policy]))

Export Policy: dept_to_as2
Definition: ['route-map dept_to_as2 permit 100\n', ' match ip address 102\n', ' set metric 50\n', ' set community 65001:2 additive\n']


### View BGP Session Status

Now that we have seen the configuration for each peer on `as2dept1`, let's ensure that their configuration is compatible with their peers

In [9]:
# The bgpSessionCompatibility question allows you to ensure that BGP sessions are compatibly configured, so that if there is IP reachability between the peers
# the sessions will be established. Compatiblity checks that the remote-as matches up on both ends, the correct update source is specified, etc...

bgpSessCompat = bfq.bgpSessionCompatibility(nodes='as2dept1').answer().frame()
bgpSessCompat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Configured_Status
0,as2dept1,default,65001,None,2.34.101.4,2,as2dist1,None,2.34.101.3,EBGP_SINGLEHOP,UNIQUE_MATCH
1,as2dept1,default,65001,None,2.34.201.4,2,as2dist2,None,2.34.201.3,EBGP_SINGLEHOP,UNIQUE_MATCH


Both of the configured BGP peers on `as2dept1` are compatible. We know that since the `Configured_Status` is `UNIQUE_MATCH`. So now let's check if they are established.

In [10]:
bgpSessStat = bfq.bgpSessionStatus(nodes='as2dept1').answer().frame()
bgpSessStat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Established_Status
0,as2dept1,default,65001,None,2.34.101.4,2,as2dist1,None,2.34.101.3,EBGP_SINGLEHOP,ESTABLISHED
1,as2dept1,default,65001,None,2.34.201.4,2,as2dist2,None,2.34.201.3,EBGP_SINGLEHOP,ESTABLISHED


Both sessions are established. Let's see if there are any configured BGP sessions, on any other device in the network, that are not established

In [23]:
bgpSessStat = bfq.bgpSessionStatus().answer().frame()
bgpSessStat[bgpSessStat['Established_Status'] != 'ESTABLISHED']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Established_Status
1,as1border1,default,1,None,None,666,None,None,3.2.2.2,EBGP_SINGLEHOP,NOT_COMPATIBLE
2,as1border1,default,1,None,None,555,None,None,5.6.7.8,EBGP_SINGLEHOP,NOT_COMPATIBLE
6,as1border2,default,1,None,10.14.22.1,4,None,None,10.14.22.4,EBGP_SINGLEHOP,NOT_COMPATIBLE
9,as2border1,default,2,None,2.1.1.1,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED
12,as2border2,default,2,None,2.1.1.2,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED
15,as2core1,default,2,None,2.1.2.1,2,as2border1,None,2.1.1.1,IBGP,NOT_ESTABLISHED
16,as2core1,default,2,None,2.1.2.1,2,as2border2,None,2.1.1.2,IBGP,NOT_ESTABLISHED
17,as2core1,default,2,None,2.1.2.1,2,as2dist1,None,2.1.3.1,IBGP,NOT_ESTABLISHED
18,as2core1,default,2,None,2.1.2.1,2,as2dist2,None,2.1.3.2,IBGP,NOT_ESTABLISHED
25,as2dist1,default,2,None,2.1.3.1,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED


Wow, there are a lot of sessions that are configured, but not established. Let's dig into these issues. Let's find all of the sessions that are not compatibly configured. For that we are looking for sessions which are not either a `UNIQUE_MATCH` or `DYNAMIC_MATCH`. The latter is for peers that are configured with a `listen-range`.

## Debug sessions that are NOT_COMPATIBLE

In [25]:
bgpSessCompat = bfq.bgpSessionCompatibility().answer().frame()
bgpSessCompat[~bgpSessCompat['Configured_Status'].isin(["UNIQUE_MATCH", "DYNAMIC_MATCH"])]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Configured_Status
1,as1border1,default,1,None,None,666,None,None,3.2.2.2,EBGP_SINGLEHOP,NO_LOCAL_IP
2,as1border1,default,1,None,None,555,None,None,5.6.7.8,EBGP_SINGLEHOP,NO_LOCAL_IP
6,as1border2,default,1,None,10.14.22.1,4,None,None,10.14.22.4,EBGP_SINGLEHOP,UNKNOWN_REMOTE
13,as2border2,default,2,None,None,2,None,None,2.1.2.2,IBGP,LOCAL_IP_UNKNOWN_STATICALLY


We see 4 entries in this table even though we only saw 3 BGP session with thes status NOT_COMPATIBLE. This means that despite not having a `Configured_Status` of `UNIQUE_MATCH` or `DYNAMIC_MATCH` one of these sessions was indeed established. This typically occurs if the mis-configuration is such that the session can ONLY be established when initiated from one side, but not the other. The likely candidate in this output is the `as2border2` session to `2.1.2.2` that has status of `LOCAL_IP_UNKNOWN_STATICALLY`.

### Debug UNKNOWN_REMOTE peer on as1border2

Batfish deems the BGP peer `10.14.22.4` is not compatible because it cannot find a device in the snapshot that has that IP address configured on any interface. That is why the status is `UNKNOWN_REMOTE`.
This will occur in  most networks, since you will not have the configurations of the devices for your external peers (ISPs, content partners, etc...). We can easily verify this by checking the output of `bfq.ipOwners`

In [14]:
# check if there is a node in the network that has the `10.14.22.4` on an interface
ipOwn = bfq.ipOwners().answer().frame()
ipOwn[ipOwn['IP']=='10.14.22.4']

,Node,VRF,Interface,IP,Mask,Active


### Debug LOCAL_IP_UNKNOWN_STATICALLY on as2border2

Now, let's dig into the sessions that are `LOCAL_IP_UNKNOWN_STATICALLY`.

An iBGP session will have the status `LOCAL_IP_UNKNOWN_STATICALLY` if you are missing the `update-source` command.

So, in this case, it is likely that the issue is that `as2border2` is missing the `update-source` command on it's BGP session. This is needed for iBGP sessions to ensure the peers pick the correct IP address to use when trying to establish the TCP session. We can find out the target remote node by looking at the `bfq.ipOwners` output

In [26]:
#find the devces that own these ip addresses
ipOwn[ipOwn['IP'].isin(['2.1.2.2'])]

,Node,VRF,Interface,IP,Mask,Active
22,as2core2,default,Loopback0,2.1.2.2,32,True


So the sessions is supposed to be between `as2border2` and `as2core2`. Let's see if this sessions established.

In [16]:
# check if either direction of the session can be established
bgpSessStat[((bgpSessStat['Local_IP']=='2.1.1.2') & (bgpSessStat['Remote_IP']=='2.1.2.2')) | ((bgpSessStat['Local_IP']=='2.1.2.2') & (bgpSessStat['Remote_IP']=='2.1.1.2'))]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Established_Status
20,as2core2,default,2,None,2.1.2.2,2,as2border2,None,2.1.1.2,IBGP,ESTABLISHED


This confirms that our theory. We are missing the `update-source` command on `as2border2` which prevents it from initiating the BGP session. But since `as2core2` is properly configured, the session is established. 

### Debug NO_LOCAL_IP on as1border1

In [28]:
bgpSessCompat[(~bgpSessCompat['Configured_Status'].isin(["UNIQUE_MATCH", "DYNAMIC_MATCH"])) & (bgpSessCompat['Node']=='as1border1')]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Configured_Status
1,as1border1,default,1,None,None,666,None,None,3.2.2.2,EBGP_SINGLEHOP,NO_LOCAL_IP
2,as1border1,default,1,None,None,555,None,None,5.6.7.8,EBGP_SINGLEHOP,NO_LOCAL_IP


For an `EBGP_SINGLEHOP` session to have `CONFIGURED_STATUS` of `NO_LOCAL_IP` this typically means that no interface exists on the box which is in the same subnet as the BGP peer's IP address. This can easily be checked by looking at the IP addresses configued on `as1border1`.

In [30]:
ipOwn[ipOwn['Node']=='as1border1']

,Node,VRF,Interface,IP,Mask,Active
7,as1border1,default,GigabitEthernet1/0,10.12.11.1,24,True
14,as1border1,default,GigabitEthernet0/0,1.0.1.1,24,True
19,as1border1,default,Loopback0,1.1.1.1,32,True


As you can see there are no interfaces with addresses that would be in the same subnet as `3.2.2.2` and `5.6.7.8`. There could be many explanations for this - 1) Peers configured ahead of the circuit connecting the neighbors is ready, 2) incorrect BGP peer address, 3) The interfaces used to exist but were decommissioned, however the BGP config was not cleaned up, 4) should be an eBGP multi-hop session with an explicit update source,  etc...

Batfish determines the Local_IP for each BGP session, either based on explicit configuration with `updates-source foo` (or equivalent non-IOS command) for iBGP sessions or eBGP multi-hop sessions,
or by determining the interface the router will use to send packets towards the BGP peer. The latter method requires the route to the peer to be known.

So, if there is no route to the configured peer, or the configured peer does not exist in the snapshot, you will see this status. We can check the output of `bfq.routes` and `bfq.ipOwners` questions to dig into this

In [31]:
bad_bgp_peer = ['3.2.2.2', '5.6.7.8']

In [32]:
ipOwn[ipOwn['IP'].isin(bad_bgp_peer)]

,Node,VRF,Interface,IP,Mask,Active
10,as3border2,default,Loopback0,3.2.2.2,32,True


So we can see that `5.6.7.8` does not exist in the network. This either means there is a mis-configuration, or the device is just not expected to be in the snapshot.

Now let's dig into the peer `3.2.2.2`, which we know is the Loopback interface on `as3border2`

In [33]:
routes = bfq.routes(network='3.2.2.2/32').answer().frame()
routes[routes['Node']=='as1border1']

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


The specific /32 is not present on `as1border1`. What about other routers?

In [34]:
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,as3border1,default,3.2.2.2/32,as3core1,3.0.1.2,dynamic,ospf,3,110,None
1,as3border2,default,3.2.2.2/32,None,AUTO/NONE(-1l),Loopback0,connected,0,0,None
2,as3core1,default,3.2.2.2/32,as3border2,3.0.2.1,dynamic,ospf,2,110,None


We can see that this route does not leave `as3`, which is why `as1border1` is unable to established the configured BGP session.
This session should have been configured as an eBGP multi-hop session with static routes pointing to the appropriate interface and next-hop

## Debugging BGP sessions that are NOT_ESTABLISHED

We have root-caused the `NOT_COMPATIBLE` sessions, now let's dig into the ones that are `NOT_ESTABLISHED`.

In [40]:
bgpSessStat[bgpSessStat['Established_Status'] == 'NOT_ESTABLISHED']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Established_Status
9,as2border1,default,2,None,2.1.1.1,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED
12,as2border2,default,2,None,2.1.1.2,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED
15,as2core1,default,2,None,2.1.2.1,2,as2border1,None,2.1.1.1,IBGP,NOT_ESTABLISHED
16,as2core1,default,2,None,2.1.2.1,2,as2border2,None,2.1.1.2,IBGP,NOT_ESTABLISHED
17,as2core1,default,2,None,2.1.2.1,2,as2dist1,None,2.1.3.1,IBGP,NOT_ESTABLISHED
18,as2core1,default,2,None,2.1.2.1,2,as2dist2,None,2.1.3.2,IBGP,NOT_ESTABLISHED
25,as2dist1,default,2,None,2.1.3.1,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED
28,as2dist2,default,2,None,2.1.3.2,2,as2core1,None,2.1.2.1,IBGP,NOT_ESTABLISHED


The  reasons for a session that is compatiable (`UNIQUE_MATCH` or `DYNAMIC_MATCH`) to not get established would be 1) missing routes or 2) some ACL in the path blocking traffic. Let's check the routing tables on `as2core1`

In [48]:
routes = bfq.routes(nodes='as2core1').answer().frame()
routes[routes['Network']=='2.1.1.1/32']

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
2,as2core1,default,2.1.1.1/32,as2border1,2.12.11.1,dynamic,ospf,2,110,None


As we can see `as2core1` has a route to it's configured neighbor `2.1.1.1`. What about the other routers?

In [49]:
routes=bfq.routes(nodes='as2border1').answer().frame()
routes[routes['Network']=='2.1.2.1/32']

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


`as2border1` does not have a route to the loopback address `2.1.2.1` of `as2core1`. Let's also check `as2border2`.

In [50]:
routes=bfq.routes(nodes='as2border2').answer().frame()
routes[routes['Network']=='2.1.2.1/32']

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


Neither border router has a route to the loopback of `as2core1`. Since the loopback is supposed to be distributed via OSPF, next step is to look at the OSPF configuration on `as2core1`

From the snippet of the configuration of `as2core1`, we can see that the Loopback address isn't part of the OSPF process:
```
interface Loopback0
 ip address 2.1.2.1 255.255.255.255

router ospf 1
 router-id 2.1.2.1
 !network 2.0.0.0 0.255.255.255 area 1
 network 2.12.0.0 0.0.255.255 area 1
 network 2.23.0.0 0.0.255.255 area 1
 ```
 
 This explains why the BGP session wasn't established.


We have finished debugging all of the sessions that were NOT_ESTABLISHED.

### Now let's look at how Batfish treats dynamic BGP sessions configured using a listen-range

We identify routers that are configured with dynamic BGP sessions by looking for `DYNAMIC_MATCH` in the `bfq.bgpSessionCompatibility` question

In [51]:
bgpSessCompat[bgpSessCompat['Configured_Status']=='DYNAMIC_MATCH']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Configured_Status
27,as2dist1,default,2,None,AUTO/NONE(-1l),65001,as2dept1,None,2.34.101.4,EBGP_SINGLEHOP,DYNAMIC_MATCH
30,as2dist2,default,2,None,AUTO/NONE(-1l),65001,as2dept1,None,2.34.201.4,EBGP_SINGLEHOP,DYNAMIC_MATCH


Let's see if any sessions were established between `2.34.101.4` and other routers in the snapshot

In [52]:
bgpSessStat[(bgpSessStat['Remote_IP']=='2.34.101.4') | (bgpSessStat['Local_IP']=='2.34.101.4')]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Established_Status
23,as2dept1,default,65001,None,2.34.101.4,2,as2dist1,None,2.34.101.3,EBGP_SINGLEHOP,ESTABLISHED
27,as2dist1,default,2,None,AUTO/NONE(-1l),65001,as2dept1,None,2.34.101.4,EBGP_SINGLEHOP,ESTABLISHED


As we can see we found a session between `as2dist1` and `as2dept1` that matches the configured dynamic BGP session

Now let's take a look at the sessions between 2.34.201.4 and other routers in the snapshot

In [53]:
bgpSessStat[(bgpSessStat['Remote_IP']=='2.34.201.4') | (bgpSessStat['Local_IP']=='2.34.201.4')]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Session_Type,Established_Status
24,as2dept1,default,65001,None,2.34.201.4,2,as2dist2,None,2.34.201.3,EBGP_SINGLEHOP,ESTABLISHED
30,as2dist2,default,2,None,AUTO/NONE(-1l),65001,as2dept1,None,2.34.201.4,EBGP_SINGLEHOP,ESTABLISHED


As you can see, both dynamic BGP sessions were established.

On that note, we conclude this notebook. We hope you found this notebook useful and informative. 


With Batfish you can easily retrieve information about BGP configuration of all devices and peers, as well as status of each peer. This enables you to easily debug any broken BGP sessions in the network.


Future notebooks will dive into more advanced topics like path analysis, debugging ACLs and firewall rules, validating routing policy, etc.. so stay tuned! 

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.